In [1]:
import gradio as gr
from transformers import pipeline
import numpy as np
import librosa
import time

#transcriber = pipeline("automatic-speech-recognition", model="asapp/sew-d-tiny-100k-ft-ls100h")
transcriber = pipeline(
    task="zero-shot-audio-classification", model="laion/clap-htsat-unfused"
)
candidate_labels = ["Sound of a hand hitting a desk", "Sound of a clap", "Sound of random noise"]

def transcribe(audio):
    start=time.time()
    sr, y = audio
    y = y.astype(np.float32)
    if len(y.shape)>=2: y = np.mean(y, axis=1)
    y /= np.max(np.abs(y))
    num_seconds = 5
    num_samples = len(y)
    last_five_seconds_start_index = num_samples - (num_seconds * sr)
    last_five_seconds_end_index = num_samples
    last_five_seconds = []
    for i in range(last_five_seconds_start_index, last_five_seconds_end_index): 
        last_five_seconds.append(y[i])
    return transcriber(last_five_seconds, candidate_labels=candidate_labels)


demo = gr.Interface(
    transcribe,
    gr.Audio(sources=["microphone"]),
    "text",
)

demo.launch()


pytorch_model.bin:   0%|          | 0.00/615M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


(127007,)
12132


In [1]:
import gradio as gr
from transformers import pipeline
import numpy as np

transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(stream, new_chunk):
    """sr, y = new_chunk
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))

    if stream is not None:
        stream = np.concatenate([stream, y])
    else:
        stream = y
    return stream, transcriber({"sampling_rate": sr, "raw": stream})["text"]"""
    sr, y = new_chunk
    if len(y.shape) > 1: y = np.mean(y,axis=1)
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    if stream is not None:
        stream = np.concatenate([stream, y])
    else:
        stream = y
    #librosa.resample(y, orig_sr=sr, target_sr=16000)
    return stream, transcriber({"sampling_rate": sr, "raw": stream})["text"]


demo = gr.Interface(
    transcribe,
    ["state", gr.Audio(sources=["microphone"], streaming=True)],
    ["state", "text"],
    live=True,
)

demo.launch()


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
